| Variable       | Value                                                                 |
|----------------|----------------------------------------------------------------------|
| **2m Temperature** | Daily values at 09:00                                             |
| **Years**      | 1940 – 2025                                                          |
| **Months**     | Jan, Feb, Mar, Apr, May, Jun, Jul, Aug, Sep, Oct, Nov, Dec           |
| **Days**       | 01, 02, 03, 04, 05, 06, 11, 13, 14, 15, 16, 17, 18, 19, 21, 22, 23, 25, 27 |
| **Time**       | 09:00                                                                |


When requesting for 24 hours, it exceeded the request limit in netcdf, so the date has been randomly selected. Dataset for remaining days has been approved and is in the process of downloading.

In [1]:
import xarray as xr

In [2]:
ds = xr.open_dataset("../dataset/tref_.nc")

### NetCDF Dataset Overview: Temperature at 2m height

| Item          | Description |
|---------------|-------------|
| **Size**      | 81 GB |
| **Dimensions** | `valid_time` (19,538 steps, 1940–2025) <br> `latitude` (721 points, 90° → -90°, 0.25degree grid spacing) <br> `longitude` (1,440 points, 0° → 359.75°, 0.25degree grid spacing) |
| **Coordinates** | `valid_time` → time values <br> `latitude` → north–south grid <br> `longitude` → east–west grid <br> `number`, `expver` → metadata |
| **Data Variable** | `t2m` → 2-meter temperature - temp at 2m from surgace |
| **Attributes** | Source: ECMWF (European Centre for Medium-Range Weather Forecasts) <br> Format: CF-1.7 (Climate/Forecast conventions) <br> Created from GRIB file via cfgrib |


In [3]:
import xarray as xr
import pandas as pd



#ktm approx bounds
lat_min, lat_max = 27.50, 28.000
lon_min, lon_max = 85.00, 86.000

ds_kathmandu = ds.sel(
    latitude=slice(lat_max, lat_min),  # slice from high to low latitude
    longitude=slice(lon_min, lon_max)  # slice from low to high longitude
)

df = ds_kathmandu.to_dataframe().reset_index()
df.to_csv("../dataset/kathmandu_valley_t2m.csv", index=False)



Value Check after conversion

In [4]:
import xarray as xr
import pandas as pd


#small set of data to check
lat_check = 27.75
lon_check = 85.25
time_check = "1990-01-01 09:00:00"

# netcdf value
nc_value = ds["t2m"].sel(latitude=lat_check, longitude=lon_check, valid_time=time_check).values
print("NetCDF t2m:", nc_value)


# Filter the same point
csv_value = df[
    (df["latitude"] == lat_check) &
    (df["longitude"] == lon_check) &
    (df["valid_time"] == time_check)
]["t2m"].values[0]
print("CSV t2m:", csv_value)

# 4. Compare
if nc_value == csv_value:
    print("Value matches Ok")
else:
    print("Different")


NetCDF t2m: 287.1366
CSV t2m: 287.1366
Value matches Ok


In [5]:
print(df.head())


           valid_time  latitude  longitude         t2m  number expver
0 1940-01-01 09:00:00      28.0      85.00  285.408539       0   0001
1 1940-01-01 09:00:00      28.0      85.25  283.617523       0   0001
2 1940-01-01 09:00:00      28.0      85.50  279.180023       0   0001
3 1940-01-01 09:00:00      28.0      85.75  274.916351       0   0001
4 1940-01-01 09:00:00      28.0      86.00  271.855804       0   0001


In [6]:
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293070 entries, 0 to 293069
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   valid_time  293070 non-null  datetime64[ns]
 1   latitude    293070 non-null  float64       
 2   longitude   293070 non-null  float64       
 3   t2m         293070 non-null  float32       
 4   number      293070 non-null  int64         
 5   expver      293070 non-null  object        
dtypes: datetime64[ns](1), float32(1), float64(2), int64(1), object(1)
memory usage: 12.3+ MB
None


In [7]:
print(df['t2m'].describe())

count    293070.000000
mean        291.967468
std           6.299188
min         264.428436
25%         288.160782
50%         292.889709
75%         296.533585
max         312.534515
Name: t2m, dtype: float64


In [8]:
df.drop(columns=['number', 'expver'], inplace=True)

Why Temp at 2m? <br><br> Because, 2-meter air temperature is an important measure for analyzing extreme weather and how water and energy move between the land surface and the atmosphere (Ullah et al., 2023).

In [9]:
print(df['latitude'].unique())
print(df['longitude'].unique())

[28.   27.75 27.5 ]
[85.   85.25 85.5  85.75 86.  ]


In [11]:
df['t2m_C'] = df['t2m'] - 273.15

In [12]:
df['t2m_C'].max()

39.38452